<a href="https://colab.research.google.com/github/LuigiAjello/Programacao_linear/blob/main/3b2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# devlab 3b2 – Método de resolução de problemas de transporte
# usando algoritmo em Python e a biblioteca PuLP
# (adaptação do exemplo em R do Prof. Bruno Miranda)

# pip install pulp
from pulp import *

# -------------------------------------------------------------------
# 1. Dados básicos
# -------------------------------------------------------------------
# Fábricas (oferta) e mercados (demanda)
fabricas  = ["F1", "F2", "F3"]
mercados  = ["M1", "M2", "M3", "M4"]

# Oferta de cada fábrica
oferta = {
    "F1": 7,
    "F2": 9,
    "F3": 18
}

# Demanda de cada mercado
demanda = {
    "M1": 5,
    "M2": 8,
    "M3": 7,
    "M4": 14
}

# Custos de transporte (linha = fábrica, coluna = mercado)
custos = [
    [19, 30, 50, 10],   # Fábrica F1
    [70, 30, 40, 60],   # Fábrica F2
    [40,  8,  7, 20]    # Fábrica F3
]

# Converte a lista em dicionário no formato PuLP
matriz_custos = makeDict([fabricas, mercados], custos, 0)

# Lista com todas as rotas possíveis (pareado fábrica-mercado)
rotas = [(f, m) for f in fabricas for m in mercados]

# -------------------------------------------------------------------
# 2. Modelo de otimização
# -------------------------------------------------------------------
model = LpProblem("Problema_de_transporte", LpMinimize)

# Variáveis de decisão: quantidade enviada em cada rota (inteira ≥ 0)
x = LpVariable.dicts("Fluxo", (fabricas, mercados), lowBound=0, cat="Integer")

# Função-objetivo: minimizar o custo total de transporte
model += lpSum(x[f][m] * matriz_custos[f][m] for (f, m) in rotas)

# Restrição de oferta: cada fábrica só pode enviar até a oferta disponível
for f in fabricas:
    model += lpSum(x[f][m] for m in mercados) == oferta[f], \
             f"Oferta_{f}"

# Restrição de demanda: cada mercado deve receber pelo menos a demanda
for m in mercados:
    model += lpSum(x[f][m] for f in fabricas) == demanda[m], \
             f"Demanda_{m}"

# -------------------------------------------------------------------
# 3. Resolver e mostrar resultados
# -------------------------------------------------------------------
model.solve()

print("Status:", LpStatus[model.status])
for v in model.variables():
    print(f"{v.name} = {v.varValue}")

if model.status == 1:          # 1 → Ótimo encontrado
    print("Custo total mínimo:", value(model.objective))
else:
    print("O modelo não foi resolvido com sucesso.")
